In [ ]:
#!pip install torch torchtext sentencepiece datasets
# Try opus books dataset for translation - https://huggingface.co/datasets/opus_books

In [30]:
import numpy as np
import torch
from torch import nn
import functorch
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 1
SP_VOCAB_SIZE = 1000
CHUNK_LENGTH = 10
Y_CHUNK_LENGTH = 5
TRAIN_SIZE = 500

In [31]:
from text_data import CNNDatasetWrapper

class Wrapper(CNNDatasetWrapper):
    split_lengths = [TRAIN_SIZE, math.floor(TRAIN_SIZE * .1), 100]
    x_length = CHUNK_LENGTH
    target_length = Y_CHUNK_LENGTH

wrapper = Wrapper(SP_VOCAB_SIZE, DEVICE)

datasets = wrapper.generate_datasets(BATCH_SIZE)
train = datasets["train"]

Found cached dataset cnn_dailymail (/Users/vik/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00, 149.58it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=cnn_dailymail --vocab_size=1000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: cnn_dailymail
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_pie

In [32]:
class GRUCell(nn.Module):
    def __init__(self, input_units, hidden_units, output_units):
        super(GRUCell, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.output_units = output_units

        k = math.sqrt(1/hidden_units)
        self.input_weights = nn.Parameter(torch.rand(3, input_units, hidden_units) * 2 * k - k)
        self.input_biases = nn.Parameter(torch.rand(3, 1, hidden_units) * 2 * k - k)

        self.hidden_weights = nn.Parameter(torch.rand(3, hidden_units, hidden_units) * 2 * k - k)
        self.hidden_biases = nn.Parameter(torch.rand(3, 1, hidden_units) * 2 * k - k)

        self.output_weight = nn.Parameter(torch.rand(hidden_units, output_units) * 2 * k - k)
        self.output_bias = nn.Parameter(torch.rand(1, output_units) * 2 * k - k)

    def forward(self, x, prev_hidden):
        # Compute the regular RNN forward pass
        # Compute update and reset gates for GRU
        reset_gate = torch.sigmoid(x @ self.input_weights[0,] + self.input_biases[0,] + prev_hidden @ self.hidden_weights[0,] + self.hidden_biases[0,])
        update_gate = torch.sigmoid(x @ self.input_weights[1,] + self.input_biases[1,] + prev_hidden @ self.hidden_weights[1,] + self.hidden_biases[1,])
        new_gate = torch.tanh(x @ self.input_weights[2,] + self.input_biases[2,] + torch.mul(reset_gate, prev_hidden @ self.hidden_weights[2,] + self.hidden_biases[2,]))

        hidden_x = torch.mul((1 - update_gate), new_gate) + torch.mul(update_gate, new_gate)

        # Compute output based on hidden state
        output_y = hidden_x @ self.output_weight + self.output_bias
        return output_y, hidden_x

In [46]:
class EncoderDecoder(nn.Module):
    def __init__(self, in_sequence_len, out_sequence_len, embedding_len, hidden_units=512, layers=2):
        super(EncoderDecoder, self).__init__()
        self.in_sequence_len = in_sequence_len
        self.out_sequence_len = out_sequence_len
        self.hidden_units = hidden_units
        self.embedding_len = embedding_len
        self.layers = layers

        self.embedding = nn.Embedding(embedding_len, hidden_units)
        self.encoders = nn.ModuleList([GRUCell(input_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])
        self.decoders = nn.ModuleList([GRUCell(input_units=hidden_units * 2, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])

        self.linear = nn.Linear(in_features=hidden_units, out_features=embedding_len)

    def attention(self, context, prev_hidden, batch_size):
        # Swap axes so the first dimension of context_attn is batch
        context_attn = torch.bmm(context.swapaxes(0,1), self.context_attention_weight.unsqueeze(0).expand(batch_size,-1,-1))
        # Swap back since prev_hidden is by batch.  This makes the first dim of cross sequence
        cross = torch.tanh(context_attn.swapaxes(0,1) + prev_hidden @ self.hidden_attention_weight)
        # This will be of dimension batch, sequence_length, 1
        attention = torch.bmm(cross.swapaxes(0,1), self.attention_weight.T.unsqueeze(0).expand(batch_size, -1, -1))
        # Drop the last singleton dimension
        attention = attention.squeeze(2)
        # Softmax the predictions across each batch
        probs = torch.softmax(attention, 1)
        diagonalized_probs = self.batched_diag(probs)
        positional_contexts = torch.sum(torch.bmm(diagonalized_probs, context.swapaxes(0,1)), dim=1).reshape(batch_size, self.input_units)
        return positional_contexts

    def forward(self, x, y):
        batch_size = x.shape[0]
        # Move batch to the second dimension, so sequence comes first
        y = y.swapaxes(0,1)
        # Embed the input sequence to reduce dimensionality
        embedded = self.embedding(x).swapaxes(0,1)

        # Encode the input sequence
        # Both tensors will have sequence then batch
        enc_hiddens = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        enc_outputs = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(self.in_sequence_len):
            seq_enc_hiddens = torch.zeros((1, batch_size, self.hidden_units), device=DEVICE)
            seq_enc_outputs = embedded[j,:,:].unsqueeze(0).to(DEVICE)
            for i in range(self.layers):
                output, hidden = self.encoders[i](seq_enc_outputs[i,], enc_hiddens[j,i])
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                output = output.unsqueeze(0)
                seq_enc_outputs = torch.cat((seq_enc_outputs, output), dim=0)
                seq_enc_hiddens = torch.cat((seq_enc_hiddens, hidden), dim=0)

            enc_hiddens = torch.cat((enc_hiddens, seq_enc_hiddens[1:].unsqueeze(0)), dim=0)
            enc_outputs = torch.cat((enc_outputs, seq_enc_outputs[1:].unsqueeze(0)), dim=0)

        # Decode to the output sequence
        # Pass in context
        context = enc_hiddens[1:,-1,:,:]
        # Both tensors will have the first dimension be the sequence
        dec_hiddens = torch.zeros(1, self.layers, batch_size, self.hidden_units, device=DEVICE)
        dec_outputs = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(self.out_sequence_len):
            # Use either the actual previous y (from the input), or the generated y if the input sequence is shorter than the generation steps.
            prev_y = y[j,:,:] if y.shape[0] > j else torch.softmax(dec_outputs[j,-1,:,:], dim=1)
            # Run embedding over previous y state
            prev_y = prev_y.argmax(dim=1).int()
            prev_y = self.embedding(prev_y)

            seq_dec_hiddens = prev_y.unsqueeze(0).to(DEVICE)
            seq_dec_outputs = torch.zeros((1, batch_size, self.hidden_units), device=DEVICE)
            for i in range(self.layers):
                output, hidden = self.decoders[i](torch.cat((seq_dec_hiddens[i,], context[-1]), dim=1), dec_hiddens[j,i,],)
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                output = output.unsqueeze(0)
                seq_dec_outputs = torch.cat((seq_dec_outputs, output), dim=0)
                seq_dec_hiddens = torch.cat((seq_dec_hiddens, hidden), dim=0)

            dec_hiddens = torch.cat((dec_hiddens, seq_dec_hiddens[1:].unsqueeze(0)), dim=0)
            dec_outputs = torch.cat((dec_outputs, seq_dec_outputs[1:].unsqueeze(0)), dim=0)

        # Move batch back to axis 0
        out_hiddens = dec_hiddens[1:,-1,:,:].swapaxes(0,1)
        out_output = self.linear(dec_hiddens[1:,-1,:,:].swapaxes(0,1))
        return out_output, out_hiddens

def generate(sequence, target, wrapper):
    pred, _ = model(sequence, target[:,0,:].unsqueeze(1))
    prompts = wrapper.decode_batch(sequence.cpu())
    texts = wrapper.decode_batch(torch.argmax(pred, dim=2).cpu())
    correct_texts = wrapper.decode_batch(torch.argmax(target, dim=2).cpu())

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [47]:
from tqdm.auto import tqdm
model = EncoderDecoder(wrapper.x_length, wrapper.target_length, hidden_units=512, layers=1, embedding_len=wrapper.vocab_size).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
EPOCHS = 1000
DISPLAY_BATCHES = 8

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad()
        forced_target = prev_target
        # Alternate use of teacher forcing vs feeding back own inputs
        if np.random.randint(2) == 0:
            forced_target = prev_target[:,0,:].unsqueeze(1)
        pred, hidden = model(sequence, forced_target)
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        match_pct += torch.sum(torch.argmax(target, 2) == torch.argmax(pred, 2)) / (Y_CHUNK_LENGTH * BATCH_SIZE)

    with torch.no_grad():
        if epoch % 10 == 0:
            # Show text generated from training prompt as an example
            # Don't feed in all of the train y sequences, just the first token
            # The other y tokens will be predicted by the model and fed back in
            sents = generate(sequence[:DISPLAY_BATCHES], prev_target[:DISPLAY_BATCHES], wrapper)
            for sent in sents:
                print(sent)
            """
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            for batch, (sequence, target, prev_target) in enumerate(valid):
                # Only feed in the first token of the actual target
                pred, hidden = model(sequence, prev_target[:,0,:].unsqueeze(1))
                loss = loss_fn(pred, target)
                valid_loss += loss.item()
            print(f"Epoch {epoch} train loss: {train_loss} valid loss: {valid_loss}")
            """
            print(f"Epoch {epoch} train loss: {train_loss} match_pct: {match_pct / len(train)}")

16it [00:01,  9.66it/s]


Sen. Barack Obama has elec | trifi | Micholike Iraqi be
NEW: Searcher says | plane | Micholiniversif driver
Football pays tri | bute on the | onself northtable Leader
Sheikha Lubna wa | s first female min | , beatniversGerma vote
Rozita Swinton, 3 | 3, was | race drajor Myanmar has
Democratic senators will hold short | "pro | Barcoli Che Jeff gr
Genarlow Wilson: | Accep | ,essi create second Friday
Sheriff: Possible | tornado caus | Barc U Twoldade
Epoch 0 train loss: 0.13085885159671307 match_pct: 0.0011718750465661287


16it [00:01,  9.58it/s]
16it [00:01,  9.54it/s]
16it [00:01,  9.61it/s]
16it [00:01,  9.61it/s]
16it [00:01,  9.51it/s]
16it [00:01,  9.59it/s]
16it [00:01,  9.57it/s]
16it [00:01,  9.57it/s]
16it [00:01,  9.38it/s]
16it [00:01,  9.41it/s]


Several famous songs w | ritten b | race be proceduretwee 10
NEW: Tennessee man describes di | ving to the | race Uoldic fire
NEW: White truck seen | on Santiago | race plotist0 law
At age 6, Elaine | Sonnen | race art Diana take of
"Grey's Anatomy | " actress | race plotduc expe flight
President Bush says Tony Snow | "will b | lo beam Colombia fou
Tip leads police to John Lewi | s at homeless | race be effectroup Turkey
Prince Harry describes Princess Diana as | "the be | race U bo Che
Epoch 10 train loss: 0.09902453888207674 match_pct: 0.04179687425494194


16it [00:01,  9.45it/s]
16it [00:01,  9.43it/s]
16it [00:01,  9.49it/s]
16it [00:01,  9.34it/s]
16it [00:01,  9.40it/s]
16it [00:01,  9.41it/s]
16it [00:01,  9.32it/s]
16it [00:01,  9.21it/s]
16it [00:01,  9.52it/s]
16it [00:01,  9.47it/s]


...

16it [00:01,  9.42it/s]
16it [00:01,  9.39it/s]
16it [00:01,  9.26it/s]
16it [00:01,  9.29it/s]
16it [00:01,  9.17it/s]
16it [00:01,  9.34it/s]
16it [00:01,  9.36it/s]
16it [00:01,  9.22it/s]
16it [00:01,  9.20it/s]
16it [00:01,  9.34it/s]


The company owns three major cas | ino com | race plot8 Barcld
Dalai Lama: "I have | no desir | raceersetss
Judge: Heather Mills "a  | less than impress | Maners; counts
Debate will be last time | top Democrats | raceers; counts
NEW: Delta cancel | ations only | race plot chief count Michigan
NEW: President Bush says he and | first lady | racelinendan count Ha
NEW: Feinstein | , Schumer | race plot 1 count say
Four wheels of A380 super | jumbo | race plot mor count ro
Epoch 30 train loss: 0.047635375056415796 match_pct: 0.23867185413837433


16it [00:01,  9.19it/s]
16it [00:01,  9.33it/s]
16it [00:01,  9.32it/s]
16it [00:02,  7.91it/s]
16it [00:02,  7.86it/s]
16it [00:02,  7.89it/s]
16it [00:02,  7.89it/s]
16it [00:02,  7.95it/s]
16it [00:02,  7.90it/s]
16it [00:02,  7.96it/s]


Smoking ban in France extended to | bars, | race bombsial Iraq
Vick's September 13 drug | test shows | raceerstrongss
NEW: FBI: Kevin | Brown accused of | race plot 1 count say
Thousands of teens attend Christian  | rallies  | Maners effect count of
Polish diplomats | brought po | raceers; counted
Nasser al Ansari | is CEO | raceersEss
NEW: Tennessee man describes di | ving to the | race Leet Colombia floor
Samsung Group chairman | , Lee K | race Le husbandii
Epoch 40 train loss: 0.03247778268996626 match_pct: 0.30078125


16it [00:02,  7.99it/s]
16it [00:02,  7.82it/s]
16it [00:02,  7.90it/s]
12it [00:01,  7.81it/s]